# Gists for work with Launchpad

You can find Launchpad API documentation here: https://launchpad.net/+apidoc/devel.html

Gerrit API documentation is here: https://gerrit-review.googlesource.com/Documentation/rest-api-changes.html

## Initialise launchpad connection

In [2]:
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import re
import datetime


from launchpadlib.launchpad import Launchpad
from pygerrit.rest import GerritRestAPI

gerrits = ['https://review.openstack.org/', 'https://review.fuel-infra.org/']
rests = map(lambda x: GerritRestAPI(x), gerrits)

lp = Launchpad.login_with('lp-report-bot', 'production', version='devel')
prj = lp.projects['fuel']

dev_focus = prj.development_focus
cur_ms = prj.getMilestone(name='10.0')

open_statuses = ['New', 'Confirmed', 'Triaged', 'In Progress']

important_tags = ['customer-found', 'support', 'swarm-blocker', 'long-haul-testing']
excluded_tags = [
    'feature', 'covered-by-bp', 'need-bp', 'needs-bp',
    'system-tests', 'docs',
    'devops', 'fuel-ci', 'fuel-build']

area_tags = ['library', 'ui', 'fuel-python', 'system-tests', 'docs', 'devops', 'fuel-ci', 'fuel-build']

high_priority = ['Critical', 'High']
low_priority = ['Medium', 'Low', 'Wishlist']

def display_url(url):
    display(HTML("- <a href='{link}'>{link}</a>".format(link=url)))

def display_date(date, highlight_old_in_days=False):
    delta = (datetime.date.today() - datetime.datetime.strptime(date, "%Y-%m-%d").date()).days
    if highlight_old_in_days and highlight_old_in_days < delta:
        return "<span style='background-color: red; color: white'>%s</span>" % date
    return date
    
def display_bug(bug, highlight_teams=False):
    def html_tag(t):
        if t in important_tags:
            return "<b>%s</b>" % t
        if t in excluded_tags:
            return "<i>%s</i>" %t
        return t
    if highlight_teams:
        highlight_date = 3
    else:
        highlight_date = False
    if bug.bug_tasks[0].assignee is None:
        assignee_name = 'None'
    else:
        assignee_name = bug.bug_tasks[0].assignee.name
        if not bug.bug_tasks[0].assignee.is_team:
            highlight_date = 3
    if bug.bug_tasks[0].milestone is None:
        ms_name = 'None'
    else:
        ms_name = bug.bug_tasks[0].milestone.name
    display(HTML("<span style='background-color: blue; color: white'>B {id}</span> <a href={link}>{title}</a> {created}/{updated}<br>- {status} / {priority} / {assignee} / {ms}<br>- [{tags}]".format(
        id=bug.id, link=bug.web_link, title=bug.title, status=bug.bug_tasks[0].status,
        assignee=assignee_name, ms=ms_name,
        priority=bug.bug_tasks[0].importance,
        tags=' '.join(map(html_tag, bug.tags)),
        created=display_date(bug.date_created.strftime("%Y-%m-%d"), 120),
        updated=display_date(bug.date_last_updated.strftime("%Y-%m-%d"), highlight_date)
    )))
    reviews = []
    for msg in bug.messages:
        for g in gerrits:
            reviews += re.findall(g + '\d+', msg.content)
            long_reviews = re.findall(g + '#/c/\d+/', msg.content)
            for u in long_reviews:
                reviews += [u.replace('#/c/', '').rstrip('/')]

    for rev_url in set(reviews):
        [base_url, id] = rev_url.rsplit('/', 1)
        rest = GerritRestAPI(base_url)
        try:
            review = rest.get('/changes/' + id)
            display_change(base_url, review)
        except:
            display_url(rev_url)


    
def display_change(rest_url, change):
    highlight_date = False
    if change['status'] == 'NEW':
        highlight_date = 3
        
    display(HTML("- <span style='background-color: lightgreen; color: black'>R {number}</span> {project} {branch} {status} <a href='{link}'>{subject}</a> {owner} {created}/{updated}".format(
        number=change['_number'],
        link="%s/%d" % (rest_url, change['_number']),
        subject=change['subject'],
        status=change['status'],
        owner=change['owner']['name'],
        project=change['project'],
        branch=change['branch'],
        created=change['created'][:10],
        updated=display_date(change['updated'][:10], highlight_date)
    )))

bug = lp.bugs[1484080]

print "Example of bug output:"
display_bug(bug)

change = rests[0].get('/changes/%d' % 225941)
print "Example of review output:"
display_change(gerrits[0], change)

Example of bug output:


Example of review output:


KeyError: 'name'

## Find all open tasks on current series and replace them with default tasks

In [2]:
dry_run = False

for bug_task in dev_focus.searchTasks(milestone=cur_ms, status=open_statuses):
    bug = bug_task.bug
    print bug.web_link
    bt0 = bug.bug_tasks[0]
    bt1 = None
    for bt in bug.bug_tasks:
        if bt.target == dev_focus:
            bt1 = bt
    print [bt0.web_link, bt0.status, bt0.importance, bt0.milestone, bt0.assignee]
    print [bt1.web_link, bt1.status, bt1.importance, bt1.milestone, bt1.assignee]
    if bt0.target != prj:
        raise Exception("Wrong project")
    if bt1.target != dev_focus:
        raise Exception("Wrong series")
    if (
        bt0.status == bt1.status and
        bt0.importance == bt1.importance and
        bt0.milestone == bt1.milestone and
        bt0.assignee == bt1.assignee):
        print "Deleting task on series 8.0.x"
    else:
        print "Tasks mismatch"
        [bt0.status, bt0.importance, bt0.milestone, bt0.assignee] = [bt1.status, bt1.importance, bt1.milestone, bt1.assignee]
        
    if not dry_run:
        bt0.lp_save()
        print "Root task updated"
        rbt = lp.bugs[bug.id].bug_tasks[0]
        if (
            rbt.status == bt1.status and
            rbt.importance == bt1.importance and
            rbt.milestone == bt1.milestone and
            rbt.assignee == bt1.assignee):
            bt1.lp_delete()
            print "Series task deleted"
        else:
            raise Exception("Recheck failed!")

print "Done"

Done


## Find all support/swarm-blocker/long-haul-testing bugs that are hidden in excluded tags

In [151]:
search_tags = list(important_tags)
search_tags.remove('customer-found')
collection = prj.searchTasks(milestone=cur_ms, status=open_statuses, tags=search_tags)
i = 0
for bt in collection:
    if set(bt.bug.tags).intersection(excluded_tags):
        display_bug(bt.bug)
        i += 1
print "Found %s bugs" % i

Found 5 bugs


## Find all covered-by-bp bugs

In [168]:
search_tags = ['covered-by-bp', 'need-bp', 'needs-bp']
collection = prj.searchTasks(milestone=cur_ms, status=open_statuses, tags=search_tags)
for bt in collection:
    display_bug(bt.bug)
print "Found %s bugs" % len(collection)

Found 38 bugs


## Top priority bugs

In [7]:
comments = {
    1503987: 'Needs assignee from python team',
    1484008: 'Needs assignee from python team',
    1505907: 'Waiting for help from Vasilenko',
    1479862: 'Needs to be moved to library or puppet team',
    1464656: 'Check that review is going fine',
    1479815: 'Will be delivered later'
}
collection = prj.searchTasks(milestone=cur_ms, status=open_statuses, tags=important_tags)
i = 0
for bt in collection:
    if not set(bt.bug.tags).intersection(excluded_tags):
        display_bug(bt.bug, True)
        try:
            print comments[bt.bug.id]
        except KeyError:
            pass
        i += 1
print "Found %s bugs" % i

Needs to be moved to library or puppet team


Needs assignee from python team


Needs assignee from python team


Waiting for help from Vasilenko


Check that review is going fine


Found 35 bugs


## Critical/High bugs

In [121]:
search_tags = map(lambda x: "-" + x, important_tags + excluded_tags)
collection = prj.searchTasks(
    milestone=cur_ms, status=open_statuses,
    tags=search_tags, tags_combinator='All',
    importance=high_priority)
for bt in collection:
    display_bug(bt.bug, True)
print "Found %s bugs" % len(collection)

Found 87 bugs


## Medium/Low/Wishlist bugs (without tech-debt)

In [110]:
search_tags = map(lambda x: "-" + x, important_tags + excluded_tags + ['tech-debt'])
collection = prj.searchTasks(
    milestone=cur_ms, status=open_statuses,
    tags=search_tags, tags_combinator='All',
    importance=low_priority)
for bt in collection:
    display_bug(bt.bug)
print "Found %s bugs" % len(collection)

Found 370 bugs


## Medium/Low/Wishlist tech-deb

In [113]:
search_tags = map(lambda x: "-" + x, important_tags + excluded_tags) + ['tech-debt']
collection = prj.searchTasks(milestone=cur_ms, status=open_statuses, tags=search_tags, tags_combinator='All', importance=low_priority)
for bt in collection:
    display_bug(bt.bug)
print "Found %s bugs" % len(collection)

Found 93 bugs


## Bugs and reviews of particular persons

In [122]:
users = (
    "dpyzhov mmosesohn sbogatkin kgalanov mmalchuk akislitsky ikalnitsky rprikhodchenko mkwiek aroma atykhonov".split() +
    "ogelbukh sryabin ikharin smurashov yorik-sar".split() + #yorik-sar=>ytaraday
    "dklenov iponomarev vsakharov vvalyavskiy mpolenchuk asvechnikov fzhadaev".split() +
    "sbanka bkupidura dszeluga azvyagintsev pzhurba rlu asaprykin".split()) #rlu=>mrelewicz

for user in users:
    print user
    p = lp.people.getByEmail(email="%s@mirantis.com" % user)
    if p is None:
        p = lp.people[user]
    collection = p.searchTasks(milestone=cur_ms, status=open_statuses, assignee=p)
    for bt in collection:
        display_bug(bt.bug)

dpyzhov
mmosesohn


sbogatkin


kgalanov


mmalchuk
akislitsky


ikalnitsky


...

sbanka
bkupidura


dszeluga


azvyagintsev
pzhurba


rlu


asaprykin


In [148]:
import urllib, yaml
params = {
    "p": "tools/lp-reports.git",
    "a": "blob_plain",
    "f": "config/teams.yaml",
    "hb": "refs/heads/master"}
print urllib.urlopen("https://review.fuel-infra.org/gitweb?p=tools/lp-reports.git;a=blob_plain;f=config/teams.yaml;hb=refs/heads/master").read()
#teams_config

Not Found


## Graphs playground

In [ ]:
x = np.linspace(0, 2*np.pi, 200)

In [ ]:
%matplotlib inline
fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
axes.set_xlabel('x')
axes.set_ylabel('y')
axes.set_title('title')
#axes.plot(x, np.sin(x), 'r')
#axes.plot(x, np.cos(x), 'g')
axes.plot(x, np.sin(x)**2, 'b')
axes.plot(x, np.cos(x)**2, 'y')
axes.plot(x, np.abs(np.sin(x)), 'g')
axes.plot(x, np.abs(np.cos(x)), 'p')
axes.plot(x, np.sin(x)**2 + np.cos(x)**2, 'r')

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_axes([0.0, 0.0, .6, .6], polar=True)
for i in range(0, 9):
    n = 2**i
    size = np.pi / (i / 2 + 1) ** 2
    size = np.pi / n
    shift = size / 2
    t1 = np.linspace(i + 2, i+2, 5)
    for j in range (0, n):
        t = np.linspace(0 + shift+size*2*j, size*(2*j+1) + shift, 5)
        ax.plot(t, t1, color='blue', lw=3);
t1 = np.linspace(1, 1, 5)
t = np.linspace(0, np.pi, 5)
ax.plot(t, t1, color='blue', lw=3);
# fig.savefig('graph.png')

## Pandas playground

In [154]:
import pandas as pd
import numpy as p

In [10]:
bug_ids = [1454325, 1442532, 1310646]
text_fields = [
    'title', 'heat', 'message_count', 'tags', 'private', 'security_related',
    'users_affected_count', 'number_of_duplicates',
    'users_unaffected_count', 'users_affected_count_with_dupes']
person_fields = ['owner']
date_fields = ['date_created', 'date_last_updated']
collection_size_fields = ['activity_collection', 'attachments_collection', 'bug_tasks_collection',
    'bug_watches_collection', 'cves_collection' ]

bt_text_fields = ['importance', 'status', 'is_complete']
bt_person_fields = ['assignee'] #, 'owner']
bt_date_fields = ['date_assigned', 'date_closed', 'date_confirmed', 'date_created', 'date_fix_committed',
    'date_fix_released', 'date_in_progress', 'date_incomplete', 'date_left_closed', 'date_left_new',
    'date_triaged']
                                            
df = pd.DataFrame(columns=text_fields + person_fields + date_fields + map(lambda x: x + '_size', collection_size_fields))
ms_df = {}
def collect_bug(bug):
    id = bug.id
    df.loc[id] = float('nan')
    for f in text_fields:
        df.loc[id][f] = getattr(bug, f)
    for f in date_fields:
        df.loc[id][f] = getattr(bug, f)
    for f in person_fields:
        if getattr(bug, f) is None:
            df.loc[id][f] = None
        else:
            df.loc[id][f] = getattr(bug, f).name
    for f in collection_size_fields:
        df.loc[id][f + '_size'] = len(getattr(bug, f))
    for bt in bug.bug_tasks:
        if bt.milestone is None:
            ms = 'Untargeted_'
        else:
            ms = bt.milestone.name + '_'
        try:
            dfx = ms_df[ms]
        except KeyError:
            dfx = pd.DataFrame(columns=map(lambda x: ms + x, bt_text_fields + bt_person_fields))
            ms_df[ms] = dfx
        dfx.loc[id] = float('nan')
        for f in bt_text_fields:
            dfx.loc[id][ms + f] = getattr(bt, f)
        for f in bt_person_fields:
            if getattr(bt, f) is None:
                dfx.loc[id][ms + f] = None
            else:
                dfx.loc[id][ms + f] = getattr(bt, f).name

for bug_id in bug_ids:
    collect_bug(lp.bugs[bug_id])

df = pd.concat([df] + ms_df.values(), axis=1)

df[['owner', '8.0_status', 'tags', '8.0_importance', '8.0_assignee']]

,owner,8.0_status,tags,8.0_importance,8.0_assignee
1310646,xenolog,In Progress,"[l23network, non-release, tech-debt]",Medium,xenolog
1442532,dklepikov,Confirmed,"[docs, qa-agree-7.0, support]",High,evkonstantinov
1454325,apalkina,Confirmed,"[covered-by-bp, feature-progress-bar, module-a...",Medium,fuel-python


,owner,8.0_status,tags,8.0_importance,8.0_assignee
1310646,xenolog,In Progress,"[l23network, non-release, tech-debt]",Medium,xenolog


In [11]:
# Download all open bugs
collection = prj.searchTasks(milestone=cur_ms, status=open_statuses)

df = pd.DataFrame(columns=text_fields + person_fields + date_fields + map(lambda x: x + '_size', collection_size_fields))
ms_df = {}

s = len(collection)
i = 0
for bt in collection:
    i += 1
    print "%d/%d %s" % (i, s, bt.bug.id)
    collect_bug(bt.bug)

df = pd.concat([df] + ms_df.values(), axis=1)

print "Found %s bugs" % len(collection)

1/914 1493372
2/914 1496614
3/914 1499905
4/914 1501348
5/914 1504094
6/914 1506405
7/914 1252730
8/914 1262313
9/914 1263895
10/914 1286151
11/914 1306792
12/914 1333458
13/914 1349702
14/914 1350623
15/914 1354803
16/914 1391481
17/914 1398113
18/914 1404373
19/914 1410207
20/914 1423328
21/914 1423511
22/914 1427198
23/914 1427468
24/914 1428129
25/914 1434015
26/914 1435610
27/914 1435827
28/914 1439417
29/914 1439776
30/914 1440046
31/914 1442532
32/914 1445311
33/914 1446668
34/914 1447488
35/914 1447604
36/914 1447957
37/914 1450100
38/914 1451837
39/914 1454296
40/914 1455847
41/914 1460169
42/914 1464640
43/914 1466897
44/914 1468807
45/914 1469045
46/914 1470408
47/914 1470468
48/914 1470545
49/914 1470823
50/914 1470846
51/914 1471172
52/914 1472303
53/914 1472927
54/914 1473146
55/914 1474362
56/914 1474791
57/914 1475322
58/914 1475972
59/914 1476565
60/914 1476957
61/914 1477223
62/914 1477748
63/914 1478572
64/914 1478984
65/914 1479299
66/914 1479403
67/914 1479742
68/9

In [43]:
#df[['owner', 'title', '8.0_status', 'tags', '8.0_importance', '8.0_assignee']]

# teams taken from here: https://review.fuel-infra.org/gitweb?p=tools/lp-reports.git;a=blob_plain;f=config/teams.yaml;hb=refs/heads/master
teams = """
mos-ceilometer
mos-cinder
mos-glance
mos-heat
mos-horizon
mos-ironic
mos-keystone
mos-kernel-virt
mos-kernel-networking
mos-kernel-storage
mos-murano
mos-neutron
mos-nova
mos-oslo
mos-packaging
mos-puppet
mos-sahara
mos-scale
mos-swift
fuel-python
fuel-ui
fuel-library
mos-maintenance
mos-linux
mos-ceph
fuel-plugin-calico
fuel-plugin-cisco-aci
fuel-plugin-external-glusterfs
fuel-plugin-cinder-netapp
fuel-plugin-zabbix
fuel-plugins-bugs
fuel-plugins-docs
mos-lma-toolchain
fuel-partner-engineering
fuel-plugin-contrail
fuel-plugin-vmware-dvs
fuel-docs
fuel-build
fuel-ci
fuel-devops
fuel-qa
mos-qa
mos-security
fuel-security
""".split()

cols_with_people = filter(lambda x: x.count('assignee'), df.columns) + ['owner']
for id in pd.Series(df[cols_with_people].values.ravel()).unique():
    try:
        if lp.people[id].is_team:
            if not id in teams:
                teams += [id]
                print id
    except:
        print "E: %s" % id

teams

E: None
fuel-octane
fuel-partner-core
mos-da
fuel-maintainers


['mos-ceilometer',
 'mos-cinder',
 'mos-glance',
 'mos-heat',
 'mos-horizon',
 'mos-ironic',
 'mos-keystone',
 'mos-kernel-virt',
 'mos-kernel-networking',
 'mos-kernel-storage',
 'mos-murano',
 'mos-neutron',
 'mos-nova',
 'mos-oslo',
 'mos-packaging',
 'mos-puppet',
 'mos-sahara',
 'mos-scale',
 'mos-swift',
 'fuel-python',
 'fuel-ui',
 'fuel-library',
 'mos-maintenance',
 'mos-linux',
 'mos-ceph',
 'fuel-plugin-calico',
 'fuel-plugin-cisco-aci',
 'fuel-plugin-external-glusterfs',
 'fuel-plugin-cinder-netapp',
 'fuel-plugin-zabbix',
 'fuel-plugins-bugs',
 'fuel-plugins-docs',
 'mos-lma-toolchain',
 'fuel-partner-engineering',
 'fuel-plugin-contrail',
 'fuel-plugin-vmware-dvs',
 'fuel-docs',
 'fuel-build',
 'fuel-ci',
 'fuel-devops',
 'fuel-qa',
 'mos-qa',
 'mos-security',
 'fuel-security',
 u'fuel-octane',
 u'fuel-partner-core',
 u'mos-da',
 u'fuel-maintainers']

In [306]:
bug = lp.bugs[1445311] # https://bugs.launchpad.net/fuel/+bug/1445311/+activity
ba = bug.activity
for a in ba:
    print a._wadl_resource.representation

{u'person_link': u'https://api.launchpad.net/devel/~alexbozhenko', u'web_link': u'https://bugs.launchpad.net/bugs/1445311/activity', u'http_etag': u'"a49c6e9caa0b60eb6eba9bc22eaaeaa54dd82593-c607ca46c77b9673130a54553f81fd2595304c96"', u'oldvalue': None, u'bug_link': u'https://api.launchpad.net/devel/bugs/1445311', u'self_link': u'https://api.launchpad.net/devel/bugs/1445311/activity', u'whatchanged': u'bug', u'message': u'added bug', u'datechanged': u'2015-04-17T03:30:01.412361+00:00', u'resource_type_link': u'https://api.launchpad.net/devel/#bug_activity', u'newvalue': None}
{u'person_link': u'https://api.launchpad.net/devel/~alexbozhenko', u'web_link': u'https://bugs.launchpad.net/bugs/1445311/activity', u'http_etag': u'"d82bb97ab5ad0332f68a486bf7b064bb56329459-c607ca46c77b9673130a54553f81fd2595304c96"', u'oldvalue': u'mos', u'bug_link': u'https://api.launchpad.net/devel/bugs/1445311', u'self_link': u'https://api.launchpad.net/devel/bugs/1445311/activity', u'whatchanged': u'affects',

In [289]:
df1 = pd.DataFrame({'Ivan': ['M', 25], 'Peter': ['M', 30], 'Oleg': ['M', 28], 'Olga': ['W', 32]}, index=['sex', 'age']).transpose()
df2 = pd.DataFrame({'Volvo': ['Peter', 2], 'Audi': ['Peter', 1], 'Nissan': ['Olga', 4]}, index=['owner', 'age']).transpose()
df2

,owner,age
Audi,Peter,1
Nissan,Olga,4
Volvo,Peter,2


In [51]:
teams_map = {}
for t in teams:
    for p in lp.people[t].members:
        try:
            teams_map[t] += [p.name]
        except KeyError:
            teams_map[t] = [p.name]

In [52]:
teams_map

{'fuel-build': [u'r0mikiam',
  u'sotpuschennikov',
  u'mrasskazov',
  u'vparakhin',
  u'dburmistrov',
  u'skulanov',
  u'kozhukalov'],
 'fuel-ci': [u'akaszuba',
  u'aevseev-h',
  u'dkaiharodsev',
  u'ibelikov',
  u'afedorova'],
 'fuel-devops': [u'heos',
  u'teran',
  u'pbrzozowski',
  u'mmatuszkowiak',
  u'skulanov',
  u'acharykov'],
 'fuel-docs': [u'evkonstantinov',
  u'aadamov',
  u'mzlatkova',
  u'ologvinova',
  u'ogusarenko',
  u'skarslioglu',
  u'tzn',
  u'fsoppelsa',
  u'dborodaenko',
  u'ipovolotskaya',
  u'eshumakher'],
 'fuel-library': [u'mmalchuk',
  u'kgalanov',
  u'zynzel',
  u'ashtokolov',
  u'alex-schultz',
  u'sbogatkin',
  u'bpiotrowski',
  u'tzn',
  u'smakar',
  u'omolchanov',
  u'sgolovatiuk',
  u'adidenko',
  u'bogdando',
  u'raytrac3r',
  u'xenolog',
  u'dborodaenko',
  u'xarses',
  u'idv1985',
  u'vkuklin'],
 u'fuel-maintainers': [u'afedorova'],
 u'fuel-octane': [u'sryabin',
  u'smurashov',
  u'yorik-sar',
  u'akscram',
  u'gelbuhos'],
 u'fuel-partner-core': [u'ipo

In [54]:
df.to_csv('open_fuel_8_0.csv', encoding='utf-8')

In [55]:
import yaml
with open('teams.yml', 'w') as outfile:
    outfile.write(yaml.dump(teams_map) )